In [ ]:
import sys
sys.path.append("../../digitech_classify")

import numpy as np
import pandas as pd
import faiss
from sklearn.preprocessing import normalize

from digitech_classify.pipeline.config import INTERIM_DATA_DIR, PROCESSED_DATA_DIR, DATA_DIR
from digitech_classify.pipeline.data_engineering.features import build_faiss_index, search_top_k, build_keyword_tag_df



In [ ]:
comp_data = np.load(INTERIM_DATA_DIR / "negative_samples_embeddings.npz", allow_pickle=True)
company_vectors = comp_data["embeddings"]
org_ids = comp_data["org_ID"]  

print(company_vectors.dtype, company_vectors.shape, company_vectors.flags['C_CONTIGUOUS'])


In [ ]:

kw_data = np.load(INTERIM_DATA_DIR / "keywords_embeddings_all-MiniLM-L6-v2.npz", allow_pickle=True)
keyword_vectors = kw_data["embeddings"]
keyword_texts = kw_data["keywords"]

print(keyword_vectors.dtype, keyword_vectors.shape, keyword_vectors.flags['C_CONTIGUOUS'])      

In [ ]:
faiss.normalize_L2(company_vectors)   
faiss.normalize_L2(keyword_vectors) 

In [ ]:

keyword_index = build_faiss_index(keyword_vectors)

In [ ]:
TOP_K = 5
D, I = search_top_k(keyword_index, company_vectors, top_k=TOP_K, batch_size=10000)
print("Similarity search shapes:", D.shape, I.shape)   # num_companies and TOP_K results

In [ ]:
tagged_df = build_keyword_tag_df(D, I, org_ids, keyword_texts, sim_threshold=0.45)
print(tagged_df.head())


In [ ]:
tagged_df.shape

In [ ]:
company_path = INTERIM_DATA_DIR / "cleaned_companies_text.csv"
descriptions_df = pd.read_csv(company_path, usecols=['org_ID','organisation_name', 'search_text'])  
print(descriptions_df.columns.to_list())

In [ ]:
tagged_df = tagged_df.merge(descriptions_df, on='org_ID', how='inner')

print(tagged_df.shape)

In [ ]:
kw_df = pd.read_excel(DATA_DIR / "keywords_combined_digital/Keywords_Combined.xlsx", sheet_name="Sheet1")
kw_df['Keyword'] = kw_df['Keyword'].astype(str).str.strip().str.lower()
kw_df = kw_df[kw_df['yes/no'] == 'yes']
kw_df = kw_df.drop(columns=['yes/no', 'Subcluster', 'Cluster'])
kw_df['Sector'] = kw_df['Sector'].astype(str).str.strip().str.lower()

In [ ]:
tagged_df = tagged_df.merge(kw_df[['Keyword', 'Sector']], left_on='keyword', right_on='Keyword', how='left')

In [ ]:
tagged_df_filtered = tagged_df[tagged_df['similarity'] > 0.60]
print(f"Filtered shape: {tagged_df_filtered.shape}")
print(f"Original shape: {tagged_df.shape}")

In [ ]:
unique_keywords = tagged_df_filtered['keyword'].unique().tolist()

print(unique_keywords)

In [ ]:
save_path = PROCESSED_DATA_DIR / "company_tagged_all-MiniLM-L6-v2.csv"
tagged_df.to_csv(save_path, index=False)